In [ ]:
import mne
import mne.io
import mne_bids
import numpy as np
import plotly.express as px
import q1k_sync_tools as qst

In [ ]:
subject_id = '0006'
task = 'ssvep'
bids_root = '.'

In [ ]:
#read EEG mff data
eeg_raw = mne.io.read_raw_egi('sourcedata/s06/eeg/s06/s06_ssvep_20230313_113128.mff/',preload=True)
eeg_raw_df = eeg_raw.to_data_frame()

#get the EEG events from the stim stim channel
eeg_events = mne.find_events(eeg_raw, stim_channel='STI 014')

#handle task specific EEG event interpretation..
eeg_events, eeg_stims, eeg_iti, eeg_event_dict = qst.eeg_event_test(eeg_events,task_name='ssvep')

In [ ]:
#read the asc eye tracking data and convert it to a dataframe
et_raw = mne.io.read_raw_eyelink ('sourcedata/s06/eyetracking/s06ssvep.asc')
et_raw_df = et_raw.to_data_frame()

#handle task specific ET event interpretation..
et_events, et_stims, et_iti = qst.et_event_test(et_raw_df,task_name='ssvep')

In [ ]:
eeg_et_raw = qst.eeg_et_combine(eeg_raw, et_raw, eeg_stims, et_stims)

In [ ]:
scalings=dict(eyegaze=1e1)
eeg_et_raw.plot(events=eeg_events,scalings=scalings)

In [ ]:
bids_path = mne_bids.BIDSPath(subject=subject_id, task=task, root=bids_root)
mne_bids.write_raw_bids(eeg_raw, bids_path, events=eeg_events, event_id=eeg_event_dict, overwrite=True, allow_preload=True, format='EDF')

In [ ]:
qst.show_sync_offsets(eeg_stims,et_stims)

In [ ]:
#plot the event latencies
fig = px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.show()

In [ ]:
#show the stimulus DIN event inter-trial intervals... this should be replicated in the eye-tracking recording..
fig = px.line(y=eeg_iti)
fig.show()

In [ ]:
fig = px.scatter(x=et_events['index'],y=et_events['DIN'])
fig.show()

In [ ]:
fig = px.scatter(x=et_events['index'],y=et_events['DIN_diff'])
fig.show()

In [ ]:
fig = px.scatter(x=et_events['index'],y=et_events['index'].diff())
fig.show()

In [ ]:
fig = px.line(y=et_iti)
fig.show()

In [ ]:
fig = px.scatter(y=eeg_et_offset)
fig.show()